# easy2edm

This notebook creates proceedings for the [International Conference on Educational Data Mining](https://educationaldatamining.org/conferences/) using reviewing data from [EasyChair](https://easychair.org).

See the [README](README.md) for how to download and structure the data.